In [35]:
%load_ext autoreload
%autoreload 2

import unicodecsv as csv
import datetime
import feedparser as fp
import newspaper
import helpers
from newspaper import Source, Article
import multiprocessing 
import pandas as pd
import numpy as np
import dask as da
from sklearn.metrics import pairwise_distances
import multiprocessing
from multiprocessing import cpu_count

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%%time
# Load GoogleNews word2vec-trained word embeddings:
# (https://stackoverflow.com/questions/42094180/spacy-how-to-load-google-news-word2vec-vectors)
import gensim
import spacy

# Path to google news vectors
google_news_path = "data/word_embeddings/GoogleNews-vectors-negative300.bin.gz"

# Load google news vecs in gensim
model = gensim.models.KeyedVectors.load_word2vec_format(google_news_path, binary=True)

# Init blank english spacy nlp object
nlp = spacy.blank('en')

# Loop through range of all indexes, get words associated with each index.
# The words in the keys list will correspond to the order of the google embed matrix
keys = []
for idx in range(3000000):
    keys.append(model.index2word[idx])

# Set the vectors for our nlp object to the google news vectors
nlp.vocab.vectors = spacy.vocab.Vectors(data=model.syn0, keys=keys)

CPU times: user 2min 59s, sys: 8.48 s, total: 3min 8s
Wall time: 3min 10s


/Users/alexander/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).


In [3]:
%%time
topic_modeling_df = pd.read_csv('data/processed/2019_02_08_with_topics.csv', index_col=0)

CPU times: user 24.9 s, sys: 11.7 s, total: 36.7 s
Wall time: 2min 38s


In [390]:
print(max(pd.to_datetime(topic_modeling_df.publish_date)))
print(min(pd.to_datetime(topic_modeling_df.publish_date)))

2019-04-02 00:00:00
2012-02-24 18:57:14


In [10]:
doc_1 = nlp(topic_modeling_df['clean_text'][0])
doc_2 = nlp(topic_modeling_df['clean_text'][1])
doc_3 = nlp(topic_modeling_df['clean_text'][2])

In [122]:
doc_1.similarity(doc_3)

0.9233324746108649

In [40]:
test_mat = np.stack([doc_1.vector, doc_2.vector, doc_3.vector])

In [87]:
test_mat

array([[ 1.23407552e-02,  3.27097550e-02,  1.53541816e-02,
         5.97806275e-02, -5.11604510e-02, -1.46879405e-02,
         1.25913471e-02, -6.19061217e-02,  7.19979778e-02,
         5.98698668e-02, -3.60981375e-02, -5.48369773e-02,
        -6.62904978e-03,  1.36261294e-02, -7.50808492e-02,
         4.07561064e-02,  2.83969548e-02,  4.98225875e-02,
        -1.74383912e-03, -3.41415629e-02, -3.01887859e-02,
         2.03090087e-02,  7.46628176e-03, -1.64851211e-02,
         4.73413877e-02,  5.49676595e-03, -6.09366782e-02,
         3.77761498e-02,  1.04446271e-02,  1.77859142e-02,
         6.75854599e-03, -2.32983641e-02, -2.88163163e-02,
        -1.48553820e-02,  2.41592359e-02, -8.83720070e-03,
         3.13260267e-03, -1.32394535e-02,  2.46355273e-02,
         3.89034636e-02,  7.02875480e-02, -2.11812146e-02,
         4.15381864e-02, -1.82093948e-03, -2.95656361e-02,
        -3.35123017e-02, -2.45068539e-02,  3.28406021e-02,
        -2.01430381e-03,  1.49084665e-02,  2.47009136e-0

In [110]:
%%time
# Vectorize texts:
vectorized_texts = nlp(topic_modeling_df['clean_text'][0]).vector
idx = 1
for text in topic_modeling_df[1:]['clean_text']:
    vector = nlp(text).vector
    vectorized_texts = np.vstack([vectorized_texts, vector])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


In [112]:
vectorized_texts.shape

(3327, 300)

In [141]:
%%time
# Calculate cosine similarity matrix (n_jobs = -1 means use all CPU cores):
cos_sim_mat = (1 - pairwise_distances(vectorized_texts, metric='cosine', n_jobs = -1))

CPU times: user 869 ms, sys: 625 ms, total: 1.49 s
Wall time: 1.76 s


In [142]:
# Check shape:
cos_sim_mat.shape

(3327, 3327)

In [147]:
# Save the similarity matrix, just in case:
np.save('data/processed/2019_02_10_cosine_similarity_matrix', cos_sim_mat)

### Calculate averages of distances, conditional on sources and topics:

In [465]:
rt_df = topic_modeling_df[topic_modeling_df['source'] == 'rt']
cnn_df = topic_modeling_df[topic_modeling_df['source'] == 'cnn']
fox_df = topic_modeling_df[topic_modeling_df['source'] == 'fox']
bbc_df = topic_modeling_df[topic_modeling_df['source'] == 'bbc']

In [526]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import numpy as np

x = np.random.randn(500)
trace1 = go.Histogram(x=cnn_df['Dominant_Topic'], opacity=0.75, name='CNN')
trace2 = go.Histogram(x=fox_df['Dominant_Topic'], opacity=0.75, name='Fox News')
trace3 = go.Histogram(x=rt_df['Dominant_Topic'], opacity=0.75, name='RT')
trace4 = go.Histogram(x=bbc_df['Dominant_Topic'], opacity=0.75, name='BBC')

data = [trace1, trace2, trace3, trace4]
layout = go.Layout(
    xaxis=dict(
        tickmode='linear',
        ticks='outside',
        tick0=0,
        dtick=1
    ),
    title='Topic Distributions by Source'
)
#fig = go.Figure(layout=layout)
fig = tools.make_subplots(rows=2, cols=2)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)
fig['layout'].update(title='Topic Distribution by Source',
                     xaxis1=dict(
                        tickmode='linear',
                        ticks='outside',
                        tick0=0,
                        dtick=1,
                         tickfont=dict(
            size=7
        )
                     ), 
                    yaxis1=dict(range=[0, 250], title='Count', 
                  dtick=25, tickfont=dict(
            size=7)),
         xaxis2=dict(
        tickmode='linear',
        ticks='outside',
        tick0=0,
        dtick=1,
                         tickfont=dict(
            size=7
        )
    ),yaxis2=dict(range=[0, 250], 
                  dtick=25, tickfont=dict(
            size=7)),
         xaxis3=dict(
        tickmode='linear',
        ticks='outside',
        tick0=0,
        dtick=1,title='Topic',
                         tickfont=dict(
            size=7
        )
    ),yaxis3=dict(range=[0, 250], title='Count', 
                  dtick=25, tickfont=dict(
            size=7)),
                     xaxis4=dict(
        tickmode='linear',
        ticks='outside',
        tick0=0,
        dtick=1,
         title='Topic',
                         tickfont=dict(
            size=7
        )
    ),yaxis4=dict(range=[0, 250], 
                  dtick=25, tickfont=dict(
            size=7)))

plotly.offline.iplot(fig, filename='basic histogram')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



In [528]:
topic_modeling_df.groupby(['Dominant_Topic']).count()

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,rss_published_parsed,rss_feedburner_origlink,paper_section_name,source,source_detail,pull_type,pull_date,clean_text,Topic_Perc_Contrib,Keywords
Dominant_Topic,,,,,,,,,,,,,,,,,,,,,
0.0,51,51,51,51,51,51,51,51,51,50,...,51,51,51,51,51,51,51,51,51,51
1.0,170,170,170,170,170,170,170,170,170,170,...,170,170,170,170,170,170,170,170,170,170
2.0,92,92,92,92,92,92,92,92,92,87,...,92,92,92,92,92,92,92,92,92,92
3.0,147,147,147,147,147,147,147,147,147,146,...,147,147,147,147,147,147,147,147,147,147
4.0,58,58,58,58,58,58,58,58,58,54,...,58,58,58,58,58,58,58,58,58,58
5.0,106,106,106,106,106,106,106,106,106,103,...,106,106,106,106,106,106,106,106,106,106
6.0,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
7.0,339,339,339,339,339,339,339,339,339,335,...,339,339,339,339,339,339,339,339,339,339
8.0,133,133,133,133,133,133,133,133,133,131,...,133,133,133,133,133,133,133,133,133,133


In [237]:
topic_1_df = topic_modeling_df[topic_modeling_df['Dominant_Topic'] == 1]  # Venezuela politics
topic_7_df = topic_modeling_df[topic_modeling_df['Dominant_Topic'] == 7]  # Crime # Try this
topic_13_df = topic_modeling_df[topic_modeling_df['Dominant_Topic'] == 13]  #?
topic_15_df = topic_modeling_df[topic_modeling_df['Dominant_Topic'] == 15]  # Healthcare
topic_17_df = topic_modeling_df[topic_modeling_df['Dominant_Topic'] == 17]  # US Politics # Try this
topic_19_df = topic_modeling_df[topic_modeling_df['Dominant_Topic'] == 19]  # Sports & Entertainment

In [220]:
(topic_1_df.groupby('source').count())

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,rss_published_parsed,rss_feedburner_origlink,paper_section_name,source_detail,pull_type,pull_date,clean_text,Dominant_Topic,Topic_Perc_Contrib,Keywords
source,,,,,,,,,,,,,,,,,,,,,
bbc,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
cnn,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
fox,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
rt,117,117,117,117,117,117,117,117,117,117,...,117,117,117,117,117,117,117,117,117,117


In [229]:
(topic_7_df.groupby('source').count())

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,rss_published_parsed,rss_feedburner_origlink,paper_section_name,source_detail,pull_type,pull_date,clean_text,Dominant_Topic,Topic_Perc_Contrib,Keywords
source,,,,,,,,,,,,,,,,,,,,,
bbc,53,53,53,53,53,53,53,53,53,53,...,53,53,53,53,53,53,53,53,53,53
cnn,93,93,93,93,93,93,93,93,93,93,...,93,93,93,93,93,93,93,93,93,93
fox,120,120,120,120,120,120,120,120,120,116,...,120,120,120,120,120,120,120,120,120,120
rt,73,73,73,73,73,73,73,73,73,73,...,73,73,73,73,73,73,73,73,73,73


In [205]:
(topic_13_df.groupby('source').count())

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,rss_published_parsed,rss_feedburner_origlink,paper_section_name,source_detail,pull_type,pull_date,clean_text,Dominant_Topic,Topic_Perc_Contrib,Keywords
source,,,,,,,,,,,,,,,,,,,,,
bbc,48,48,48,48,48,48,48,48,48,48,...,48,48,48,48,48,48,48,48,48,48
cnn,60,60,60,60,60,60,60,60,60,60,...,60,60,60,60,60,60,60,60,60,60
fox,53,53,53,53,53,53,53,53,53,49,...,53,53,53,53,53,53,53,53,53,53
rt,46,46,46,46,46,46,46,46,46,46,...,46,46,46,46,46,46,46,46,46,46


In [195]:
(topic_15_df.groupby('source').count())

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,rss_published_parsed,rss_feedburner_origlink,paper_section_name,source_detail,pull_type,pull_date,clean_text,Dominant_Topic,Topic_Perc_Contrib,Keywords
source,,,,,,,,,,,,,,,,,,,,,
bbc,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,14,14,14,14,14,14
cnn,23,23,23,23,23,23,23,23,23,23,...,23,23,23,23,23,23,23,23,23,23
fox,28,28,28,28,28,28,28,28,28,28,...,28,28,28,28,28,28,28,28,28,28
rt,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8


In [238]:
(topic_17_df.groupby('source').count())

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,rss_published_parsed,rss_feedburner_origlink,paper_section_name,source_detail,pull_type,pull_date,clean_text,Dominant_Topic,Topic_Perc_Contrib,Keywords
source,,,,,,,,,,,,,,,,,,,,,
bbc,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
cnn,104,104,104,104,104,104,104,104,104,104,...,104,104,104,104,104,104,104,104,104,104
fox,37,37,37,37,37,37,37,37,37,34,...,37,37,37,37,37,37,37,37,37,37
rt,57,57,57,57,57,57,57,57,57,57,...,57,57,57,57,57,57,57,57,57,57


In [239]:
(topic_19_df.groupby('source').count())

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,rss_published_parsed,rss_feedburner_origlink,paper_section_name,source_detail,pull_type,pull_date,clean_text,Dominant_Topic,Topic_Perc_Contrib,Keywords
source,,,,,,,,,,,,,,,,,,,,,
bbc,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,20,20
cnn,52,52,52,52,52,52,52,52,52,51,...,52,52,52,52,52,52,52,52,52,52
fox,82,82,82,82,82,82,82,82,82,75,...,82,82,82,82,82,82,82,82,82,82
rt,25,25,25,25,25,25,25,25,25,25,...,25,25,25,25,25,25,25,25,25,25


In [164]:
# Load the LDA model:
optimal_model = gensim.models.LdaModel.load('models/2019_02_08_optimal_model.model')

In [533]:
(optimal_model.show_topics(formatted=True))

[(15,
  '0.021*"health" + 0.017*"drug" + 0.015*"patient" + 0.014*"hospital" + 0.013*"doctor" + 0.013*"medical" + 0.012*"death" + 0.011*"child" + 0.009*"die" + 0.009*"accord"'),
 (12,
  '0.035*"woman" + 0.014*"white" + 0.013*"northam" + 0.012*"racist" + 0.012*"blackface" + 0.011*"state" + 0.010*"abortion" + 0.009*"statement" + 0.008*"page" + 0.008*"black"'),
 (8,
  '0.033*"company" + 0.025*"facebook" + 0.025*"user" + 0.024*"datum" + 0.019*"use" + 0.013*"app" + 0.008*"apple" + 0.008*"location" + 0.008*"bezos" + 0.007*"device"'),
 (4,
  '0.030*"film" + 0.015*"star" + 0.014*"show" + 0.007*"actor" + 0.007*"award" + 0.007*"movie" + 0.006*"viewer" + 0.006*"character" + 0.006*"director" + 0.006*"gold"'),
 (9,
  '0.022*"percent" + 0.013*"france" + 0.012*"migrant" + 0.011*"agent" + 0.010*"italian" + 0.010*"indian" + 0.009*"christian" + 0.008*"brown" + 0.008*"church" + 0.007*"client"'),
 (2,
  '0.019*"post" + 0.012*"show" + 0.011*"write" + 0.011*"twitter" + 0.011*"tweet" + 0.010*"photo" + 0.009*"

In [235]:
topic_modeling_df[topic_modeling_df['Dominant_Topic'] == 19]#['Keywords']

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,rss_feedburner_origlink,paper_section_name,source,source_detail,pull_type,pull_date,clean_text,Dominant_Topic,Topic_Perc_Contrib,Keywords
58,https://www.bbc.com/news,https://www.bbc.co.uk/news/entertainment-arts-...,David Bowie: Rare Ziggy Stardust footage haile...,[],David Bowie's performance of Starman on Top of...,[],[''],set([]),"[u'Mark Savage', u'Bbc Music Reporter']",2019-01-29 10:20:39.000,...,NotFound,bbc_news_entertainment_and_arts,bbc,bbc_news_entertainment_and_arts,custom,2019-01-31,David Bowie's performance of Starman on Top of...,19.0,0.3444,"game, team, year, season, play, sport, super_b..."
107,https://www.bbc.com/news,https://www.bbc.co.uk/news/world-us-canada-470...,Senator Rand Paul wins damages after neighbour...,[],Senator Rand Paul has been awarded more than $...,[],[''],set([]),[],2019-01-31 05:09:44.000,...,NotFound,bbc_news_home,bbc,bbc_news_home,custom,2019-01-31,Senator Rand Paul has been awarded more than $...,19.0,0.2060,"game, team, year, season, play, sport, super_b..."
209,https://www.bbc.com/news,https://www.bbc.co.uk/news/world-us-canada-470...,TV producer fired calling NFL's Tom Brady a 'k...,[],A TV producer in Pittsburgh has been fired for...,[],[''],set([]),[],2019-01-30 23:02:33.000,...,NotFound,bbc_news_us_and_canada,bbc,bbc_news_us_and_canada,custom,2019-01-31,A TV producer in Pittsburgh has been fired for...,19.0,0.3285,"game, team, year, season, play, sport, super_b..."
232,https://www.cnn.com/,https://www.cnn.com/2019/01/30/entertainment/m...,Maroon 5 letting Super Bowl show speak for itself,[],This year's halftime performers -- pop band Ma...,[],"['entertainment', 'Maroon 5 letting Super Bowl...",set([]),[u'Chloe Melas'],2019-01-30 18:54:11.000,...,https://www.cnn.com/2019/01/30/entertainment/m...,cnn_entertainment,cnn,cnn_entertainment,custom,2019-01-31,This year's halftime performers -- pop band Ma...,19.0,0.5217,"game, team, year, season, play, sport, super_b..."
235,https://www.cnn.com/,https://www.cnn.com/2019/01/30/entertainment/g...,Gwyneth Paltrow's rep disputes ski-crash suit,[],"Terry Sanderson, the skiier who claims he was ...",[],"['entertainment', ""Gwyneth Paltrow's rep dispu...",set([]),[u'Chloe Melas'],2019-01-30 19:52:20.000,...,https://www.cnn.com/2019/01/30/entertainment/g...,cnn_entertainment,cnn,cnn_entertainment,custom,2019-01-31,"Terry Sanderson, the skiier who claims he was ...",19.0,0.3206,"game, team, year, season, play, sport, super_b..."
291,https://www.cnn.com/,https://www.cnn.com/travel/article/lunar-new-y...,13 great places to celebrate Lunar New Year,[],Want to strut your stuff in the Year of the Pi...,[],"['travel', '13 great places to celebrate Lunar...",set([]),[],2018-02-09 00:39:40.000,...,https://www.cnn.com/travel/article/lunar-new-y...,cnn_most_recent,cnn,cnn_most_recent,custom,2019-01-31,Want to strut your stuff in the Year of the Pi...,19.0,0.3123,"game, team, year, season, play, sport, super_b..."
356,https://www.cnn.com/,https://bleacherreport.com/articles/2818418-ro...,Roger Goodell: NFL Teams Would Sign Colin Kaep...,[],NFL commissioner Roger Goodell has offered his...,[],"['NFL', 'Roger Goodell', 'Breaking News', 'Col...",set([]),[u'Adam Wells'],2019-01-30 20:28:42.000,...,https://bleacherreport.com/articles/2818418-ro...,cnn_top_stories,cnn,cnn_top_stories,custom,2019-01-31,NFL commissioner Roger Goodell has offered his...,19.0,0.5324,"game, team, year, season, play, sport, super_b..."
360,https://www.cnn.com/,https://www.cnn.com/2019/01/30/sport/roger-goo...,NFL on blown Saints call: 'Our officials are h...,[],Atlanta (CNN) While he admitted publicly that ...,[],"['sport', ""NFL on blown Saints call: 'Our offi...",set([]),[u'Jill Martin'],2019-01-30 20:07:51.000,...,https://www.cnn.com/2019/01/30/sport/roger-goo...,cnn_top_stories,cnn,cnn_top_stories,custom,2019-01-31,Atlanta (CNN) While he admitted publicly that ...,19.0,0.2931,"game, team, year, s

### Statistical Analysis


In [247]:
cnn_topic7_idx = topic_7_df[topic_7_df['source'] == 'cnn'].index.values
fox_topic7_idx = topic_7_df[topic_7_df['source'] == 'fox'].index.values
rt_topic7_idx = topic_7_df[topic_7_df['source'] == 'rt'].index.values

In [251]:
print(cnn_topic7_idx.shape)
print(fox_topic7_idx.shape)

(93,)
(120,)


In [294]:
def make_similarity_df(cnn_idx_arr, target_source_idx_arr, target_article_category, topic, cos_sim_mat):
    """
    """
    similarity_df = pd.DataFrame(columns=['cnn_article', 'target_article', 
                                          'target_article_category', 'topic', 'cos_similarity'])
    for cnn_idx in cnn_idx_arr:
        for target_idx in target_source_idx_arr:
            row = {'cnn_article': cnn_idx,
                   'target_article': target_idx,
                   'target_article_category': target_article_category,
                   'topic': topic,
                   'cos_similarity': cos_sim_mat[cnn_idx, target_idx]}
            similarity_df = similarity_df.append(row, ignore_index=True)
    return similarity_df

In [303]:
%%time
topic7_similarity_cnn_fox_df = make_similarity_df(cnn_idx_arr=cnn_topic7_idx, 
                                                  target_source_idx_arr=fox_topic7_idx, 
                                                  target_article_category='fox',
                                                  topic=7, 
                                                  cos_sim_mat=cos_sim_mat)
topic7_similarity_cnn_rt_df = make_similarity_df(cnn_idx_arr=cnn_topic7_idx, 
                                                  target_source_idx_arr=rt_topic7_idx, 
                                                  target_article_category='rt',
                                                  topic=7, 
                                                  cos_sim_mat=cos_sim_mat)

CPU times: user 1min 36s, sys: 259 ms, total: 1min 36s
Wall time: 1min 36s


In [309]:
master_df = topic7_similarity_cnn_fox_df.append(topic7_similarity_cnn_rt_df, ignore_index=True)

In [536]:
cnn_topic17_idx = topic_17_df[topic_17_df['source'] == 'cnn'].index.values
fox_topic17_idx = topic_17_df[topic_17_df['source'] == 'fox'].index.values
rt_topic17_idx = topic_17_df[topic_17_df['source'] == 'rt'].index.values

In [540]:
%%time
topic17_similarity_cnn_fox_df = make_similarity_df(cnn_idx_arr=cnn_topic17_idx, 
                                                  target_source_idx_arr=fox_topic17_idx, 
                                                  target_article_category='fox',
                                                  topic=17, 
                                                  cos_sim_mat=cos_sim_mat)
topic17_similarity_cnn_rt_df = make_similarity_df(cnn_idx_arr=cnn_topic17_idx, 
                                                  target_source_idx_arr=rt_topic17_idx, 
                                                  target_article_category='rt',
                                                  topic=17, 
                                                  cos_sim_mat=cos_sim_mat)

CPU times: user 47.5 s, sys: 222 ms, total: 47.7 s
Wall time: 47.8 s


In [544]:
master_df = topic7_similarity_cnn_fox_df
master_df = master_df.append(topic7_similarity_cnn_rt_df, ignore_index=True)
master_df = master_df.append(topic17_similarity_cnn_fox_df, ignore_index=True)
master_df = master_df.append(topic17_similarity_cnn_rt_df, ignore_index=True)

In [545]:
master_df = master_df.append(topic17_similarity_cnn_rt_df, ignore_index=True)

In [554]:
master_df.groupby(['topic', 'target_article_category']).describe()#.to_csv('topic7_17_cnn_rt_fox_sumstats.csv', index=False)

cos_similarity                                \
                                       count      mean       std       min   
topic target_article_category                                                
7     fox                            11160.0  0.829557  0.053827  0.546681   
      rt                              6789.0  0.839205  0.048338  0.622709   
17    fox                             3848.0  0.883838  0.053208  0.616143   
      rt                             11856.0  0.880365  0.048125  0.603045   

                                                                       
                                    25%       50%       75%       max  
topic target_article_category                                          
7     fox                      0.799794  0.835679  0.867431  0.983914  
      rt                       0.811862  0.844044  0.872818  0.970934  
17    fox                      0.859033  0.893440  0.919283  0.986398  
      rt                       0.857095  0.889097  0.914101  0.977500

In [568]:
import plotly.io as pio
fox_df = master_df[master_df['target_article_category'] == 'fox']
rt_df = master_df[master_df['target_article_category'] == 'rt']
twosample_results_fox = scipy.stats.ttest_ind(fox_df[fox_df['topic']==7].cos_similarity,
                                              fox_df[fox_df['topic']==17].cos_similarity,
                                             equal_var=False)

twosample_results_rt = scipy.stats.ttest_ind(rt_df[rt_df['topic']==7].cos_similarity,
                                              rt_df[rt_df['topic']==17].cos_similarity,
                                             equal_var=False)
fox_mean_dif = 0.883838 - 0.829557
rt_mean_dif = 0.880365 - 0.839205

matrix_twosample = [
    ['', 'Mean Difference', 'Test Statistic', 'p-value'],
    ['CNN vs Fox News', fox_mean_dif, twosample_results_fox[0], twosample_results_fox[1]],
    ['CNN vs RT', rt_mean_dif, twosample_results_rt[0], twosample_results_rt[1]]
]

twosample_table = FF.create_table(matrix_twosample, index=True)
plotly.offline.iplot(twosample_table, filename='twosample-table')
#pio.write_image(plotly.offline.iplot(twosample_table, filename='twosample-table'), 'images/fig1.png')
print(twosample_results_fox)
print(twosample_results_rt)

/Users/alexander/anaconda3/lib/python3.6/site-packages/plotly/tools.py:1545: UserWarning:

plotly.tools.FigureFactory.create_table is deprecated. Use plotly.figure_factory.create_table



Ttest_indResult(statistic=-54.40850503534507, pvalue=0.0)
Ttest_indResult(statistic=-56.03708842489034, pvalue=0.0)


In [555]:
twosample_results_fox = scipy.stats.ttest_ind(fox_topic7_15_df[fox_topic7_15_df['topic'] == 7].cos_similarity,
                                              fox_topic7_15_df[fox_topic7_15_df['topic'] == 15].cos_similarity,
                                              equal_var=False)

(0.9675233364105225, 2.802596928649634e-44)


/Users/alexander/anaconda3/lib/python3.6/site-packages/scipy/stats/morestats.py:1309: UserWarning:

p-value may not be accurate for N > 5000.



In [333]:
sources_to_cat = {'cnn': 0, 'fox': 1, 'rt': 2}

In [338]:
master_df['target_article_category_code'] = pd.Categorical(master_df.target_article_category).codes

In [362]:
fox_topic7_15_df = master_df[master_df['target_article_category'] == 'fox']
fox_topic7_15_df.head()

,cnn_article,target_article,target_article_category,topic,cos_similarity,target_article_category_code
0,230,439,fox,7,0.875211,0
1,230,441,fox,7,0.949309,0
2,230,451,fox,7,0.858029,0
3,230,467,fox,7,0.866770,0
4,230,480,fox,7,0.839578,0


In [380]:
import scipy
import plotly.plotly as py
import plotly
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
from scipy.stats import shapiro

plotly.offline.init_notebook_mode(connected=True)
print(shapiro(fox_topic7_15_df[fox_topic7_15_df['topic'] == 7].cos_similarity))

twosample_results_fox = scipy.stats.ttest_ind(fox_topic7_15_df[fox_topic7_15_df['topic'] == 7].cos_similarity,
                                              fox_topic7_15_df[fox_topic7_15_df['topic'] == 15].cos_similarity,
                                              equal_var=False)
matrix_twosample = [
    ['', 'Test Statistic', 'p-value'],
    ['Sample Data', twosample_results_fox[0], twosample_results_fox[1]]
]

twosample_table = FF.create_table(matrix_twosample, index=True)
plotly.offline.iplot(twosample_table, filename='twosample-table')

(0.9675233364105225, 2.802596928649634e-44)


/Users/alexander/anaconda3/lib/python3.6/site-packages/scipy/stats/morestats.py:1309: UserWarning:

p-value may not be accurate for N > 5000.

/Users/alexander/anaconda3/lib/python3.6/site-packages/plotly/tools.py:1545: UserWarning:

plotly.tools.FigureFactory.create_table is deprecated. Use plotly.figure_factory.create_table



In [379]:
twosample_results_fox

Ttest_indResult(statistic=12.752880048403087, pvalue=1.1026747934686911e-33)

In [323]:
import statsmodels.api as sm
#from statsmodels.api import add_constant

In [340]:
X = master_df[['target_article_category_code', 'topic']]
y = master_df['cos_similarity']

In [342]:
X

,const,target_article_category_code,topic
0,1.0,0,7
1,1.0,0,7
2,1.0,0,7
3,1.0,0,7
4,1.0,0,7
5,1.0,0,7
6,1.0,0,7
7,1.0,0,7
8,1.0,0,7
9,1.0,0,7


In [344]:
X = sm.add_constant(X)
est = sm.OLS(y, X.astype(float)).fit()

In [346]:
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         cos_similarity   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     198.8
Date:                Mon, 11 Feb 2019   Prob (F-statistic):           3.82e-86
Time:                        01:02:01   Log-Likelihood:                 28871.
No. Observations:               18777   AIC:                        -5.774e+04
Df Residuals:                   18774   BIC:                        -5.771e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                            0.8527      0.002    477.103      0.000       0.849       0.856
target_article_category_code     0.0101      0.001     12.806      0.000       0.009       0.012
topic                           -0.0033      0.000    -14.398      0.000      -0.004      -0.003
==============================================================================
Omnibus:                     1718.526   Durbin-Watson:                   1.372
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2471.063
Skew:                          -0.728   Prob(JB):                         0.00
Kurtosis:                       4.020   Cond. No.                         36.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [311]:
master_df

,cnn_article,target_article,target_article_category,topic,cos_similarity
0,230,439,fox,7,0.875211
1,230,441,fox,7,0.949309
2,230,451,fox,7,0.858029
3,230,467,fox,7,0.866770
4,230,480,fox,7,0.839578
5,230,482,fox,7,0.841156
6,230,484,fox,7,0.819665
7,230,485,fox,7,0.866371
8,230,486,fox,7,0.817849
9,230,492,fox,7,0.860888


In [304]:
topic7_similarity_cnn_fox_df

,cnn_article,target_article,target_article_category,topic,cos_similarity
0,230,439,fox,7,0.875211
1,230,441,fox,7,0.949309
2,230,451,fox,7,0.858029
3,230,467,fox,7,0.866770
4,230,480,fox,7,0.839578
5,230,482,fox,7,0.841156
6,230,484,fox,7,0.819665
7,230,485,fox,7,0.866371
8,230,486,fox,7,0.817849
9,230,492,fox,7,0.860888


In [300]:
%%time
topic7_similarity_cnn_rt_df = pd.DataFrame(columns=['cnn_article', 'target_article', 'target_article_category', 'cos_similarity'])
for cnn_idx in cnn_topic7_idx:
    for rt_idx in rt_topic7_idx:
        row = {'cnn_article': cnn_idx,
               'target_article': rt_idx,
               'target_article_category': 'rt',
               'cos_similarity': cos_sim_mat[cnn_idx, rt_idx]}
        topic7_similarity_cnn_rt_df = topic7_similarity_cnn_rt_df.append(row, ignore_index=True)

CPU times: user 29.2 s, sys: 103 ms, total: 29.3 s
Wall time: 29.4 s


In [302]:
topic7_similarity_cnn_rt_df

,cnn_article,target_article,target_article_category,cos_similarity
0,230,571,rt,0.792108
1,230,575,rt,0.842052
2,230,578,rt,0.851453
3,230,592,rt,0.871155
4,230,598,rt,0.892496
5,230,607,rt,0.878043
6,230,624,rt,0.883142
7,230,626,rt,0.903254
8,230,632,rt,0.866110
9,230,634,rt,0.880412


In [296]:
topic7_similarity_cnn_rt_df

,cnn_article,target_article,target_article_category,cos_similarity
0,230,571,rt,0.792108
1,230,575,rt,0.842052
2,230,578,rt,0.851453
3,230,592,rt,0.871155
4,230,598,rt,0.892496
5,230,607,rt,0.878043
6,230,624,rt,0.883142
7,230,626,rt,0.903254
8,230,632,rt,0.866110
9,230,634,rt,0.880412


In [290]:
topic7_similarity_cnn_fox_rt_df = topic7_similarity_cnn_fox_df.append(topic7_similarity_cnn_rt_df)

In [293]:
topic7_similarity_cnn_fox_rt_df

,cnn_article,target_article,target_article_category,cos_similarity
0,230,439,fox,0.875211
1,230,441,fox,0.949309
2,230,451,fox,0.858029
3,230,467,fox,0.866770
4,230,480,fox,0.839578
5,230,482,fox,0.841156
6,230,484,fox,0.819665
7,230,485,fox,0.866371
8,230,486,fox,0.817849
9,230,492,fox,0.860888


In [264]:
topic7_cossim_df = pd.DataFrame(data=cnn_topic7_idx, columns=['cnn_article'])
topic7_cossim_df.set_index('cnn_article', inplace=True)

In [265]:
topic7_cossim_df

""
cnn_article
230
245
258
287
305
320
344
359
402
